In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/h5-aligned/H5_GisaidData.aligned.fasta
/kaggle/input/h5-98cluster/H5_GisaidData.clustered.aligned.fasta


In [2]:
!nvidia-smi

Fri Nov  8 22:29:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install biopython

In [4]:
import Bio.SeqIO
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import date
from Bio.Align import PairwiseAligner
from Bio.Data import CodonTable
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,f_classif

# Prepare protein seqs

In [5]:
import os
import pandas as pd

# Path to your FASTA file
path = '/kaggle/input/h5-98cluster/H5_GisaidData.clustered.aligned.fasta'

# Initialize an empty list to store sequences
sequences = []

# Open the FASTA file and read sequences
with open(path, 'r') as file:
    sequence = ''
    for line in file:
        line = line.strip()
        if line.startswith('>'):
            if sequence:
                sequences.append(sequence)
                sequence = ''
        else:
            sequence += line
    if sequence:
        sequences.append(sequence)

print(f"Total sequences loaded: {len(sequences)}")

Total sequences loaded: 591


In [6]:
import numpy as np

# List of standard amino acid single-letter codes
amino_acids = [
    'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I',
    'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'
]

# Create a mapping from amino acids to integers
aa_to_int = {aa: idx + 1 for idx, aa in enumerate(amino_acids)}  # Start indexing from 1
int_to_aa = {idx + 1: aa for idx, aa in enumerate(amino_acids)}

In [7]:
# Convert sequences to integer sequences
int_sequences = []
for seq in sequences:
    int_seq = [aa_to_int.get(aa, 0) for aa in seq]  # Use 0 for unknown amino acids
    int_sequences.append(int_seq)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the maximum sequence length
max_sequence_length = max(len(seq) for seq in int_sequences)

# Pad sequences
X = pad_sequences(int_sequences, maxlen=max_sequence_length, padding='post', value=0)

print(f"Shape of X: {X.shape}")

Shape of X: (591, 648)


In [9]:
# Prepare input-output pairs
X_data = []
y_data = []

for seq in int_sequences:
    for i in range(1, len(seq)):
        X_data.append(seq[:i])
        y_data.append(seq[i])

# Pad the input sequences
X_data = pad_sequences(X_data, maxlen=max_sequence_length, padding='post', value=0)

# Convert outputs to numpy array
y_data = np.array(y_data)

print(f"Total samples: {len(X_data)}")
print(f"Shape of X_data: {X_data.shape}")
print(f"Shape of y_data: {y_data.shape}")

Total samples: 382377
Shape of X_data: (382377, 648)
Shape of y_data: (382377,)


In [10]:
from tensorflow.keras.utils import to_categorical

num_classes = len(amino_acids) + 1  # Plus one for padding/unknown

# Convert y_data to one-hot encoded vectors
y_data = to_categorical(y_data, num_classes=num_classes)

print(f"Shape of y_data after one-hot encoding: {y_data.shape}")

Shape of y_data after one-hot encoding: (382377, 21)


In [11]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_val, y_train, y_val = train_test_split(
    X_data, y_data, test_size=0.1, random_state=42
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")

Training samples: 344139
Validation samples: 38238


In [15]:
print(f"num_classes: {num_classes}")
print(f"max_sequence_length: {max_sequence_length}")

num_classes: 21
max_sequence_length: 648


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

# Define model parameters
embedding_dim = 64
lstm_units = 128

# Build the model
model = Sequential()
model.add(Embedding(
    input_dim=num_classes,
    output_dim=embedding_dim,
    input_length=max_sequence_length,
    mask_zero=True,  # Keep masking enabled
    input_shape=(max_sequence_length,)
))
model.add(Bidirectional(LSTM(
    lstm_units,
    activation='tanh',
    recurrent_activation='hard_sigmoid',  # Disable cuDNN
    return_sequences=False
)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 648, 64)        │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 256)            │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 21)             │         5,397 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 204,373 (798.33 KB)

 Trainable params: 204,373 (798.33 KB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

Epoch 1/10


I0000 00:00:1731105275.607940     117 service.cc:145] XLA service 0x77fab80db8f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731105275.607983     117 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1731105277.421418     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1345/1345 ━━━━━━━━━━━━━━━━━━━━ 271s 198ms/step - accuracy: 0.4332 - loss: 2.0623 - val_accuracy: 0.8780 - val_loss: 0.5389
Epoch 2/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 263s 195ms/step - accuracy: 0.8899 - loss: 0.4841 - val_accuracy: 0.9012 - val_loss: 0.3980
Epoch 3/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 263s 195ms/step - accuracy: 0.9053 - loss: 0.3744 - val_accuracy: 0.9104 - val_loss: 0.3382
Epoch 4/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 263s 195ms/step - accuracy: 0.9134 - loss: 0.3219 - val_accuracy: 0.9135 - val_loss: 0.3130
Epoch 5/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 263s 195ms/step - accuracy: 0.9191 - loss: 0.2926 - val_accuracy: 0.9186 - val_loss: 0.2952
Epoch 6/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 262s 195ms/step - accuracy: 0.9240 - loss: 0.2743 - val_accuracy: 0.9201 - val_loss: 0.2880
Epoch 7/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 263s 195ms/step - accuracy: 0.9267 - loss: 0.2611 - val_accuracy: 0.9213 - val_loss: 0.2804
Epoch 8/10
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 322s 196ms/step - accuracy: 0.9

In [22]:
# Evaluate on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

1195/1195 ━━━━━━━━━━━━━━━━━━━━ 31s 26ms/step - accuracy: 0.9251 - loss: 0.2677
Validation Loss: 0.2644939124584198
Validation Accuracy: 0.9249960780143738


In [25]:
# Function to predict the next amino acid given a sequence
def predict_next_amino_acid(sequence, model, aa_to_int, int_to_aa, max_sequence_length):
    # Convert sequence to integers
    int_seq = [aa_to_int.get(aa, 0) for aa in sequence]
    # Pad the sequence
    padded_seq = pad_sequences([int_seq], maxlen=max_sequence_length, padding='post', value=0)
    # Predict
    prediction = model.predict(padded_seq)
    # Get the amino acid with the highest probability
    predicted_index = np.argmax(prediction)
    predicted_aa = int_to_aa.get(predicted_index, 'Unknown')
    return predicted_aa

# Example usage
test_sequence = 'MEKIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTH'
predicted_aa = predict_next_amino_acid(test_sequence, model, aa_to_int, int_to_aa, max_sequence_length)
print(f"Given the sequence '{test_sequence}', the predicted next amino acid is '{predicted_aa}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Given the sequence 'MEKIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTH', the predicted next amino acid is 'A'


In [26]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_sequence(
    seed_sequence,
    model,
    aa_to_int,
    int_to_aa,
    max_sequence_length,
    desired_length,
    temperature=1.0
):
    """
    Generates a protein sequence of desired length.

    Parameters:
    - seed_sequence: The starting amino acid sequence (string).
    - model: Trained Keras model for sequence prediction.
    - aa_to_int: Dictionary mapping amino acids to integers.
    - int_to_aa: Dictionary mapping integers to amino acids.
    - max_sequence_length: Maximum sequence length used during training.
    - desired_length: The desired length of the generated sequence.
    - temperature: Temperature parameter for controlling randomness.

    Returns:
    - Generated amino acid sequence (string).
    """
    # Initialize the generated sequence with the seed_sequence
    generated_sequence = seed_sequence

    # Generate until the desired length is reached
    while len(generated_sequence) < desired_length:
        # Convert current sequence to integers
        int_seq = [aa_to_int.get(aa, 0) for aa in generated_sequence]
        # Pad the sequence
        padded_seq = pad_sequences(
            [int_seq],
            maxlen=max_sequence_length,
            padding='pre',
            value=0
        )
        # Predict
        predictions = model.predict(padded_seq, verbose=0)[0]
        # Adjust predictions using temperature
        predictions = np.asarray(predictions).astype('float64')
        predictions = np.log(predictions + 1e-7) / temperature  # Add small value to prevent log(0)
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        # Sample the next amino acid
        predicted_index = np.random.choice(len(predictions), p=predictions)
        predicted_aa = int_to_aa.get(predicted_index, '')
        # Append the predicted amino acid to the sequence
        generated_sequence += predicted_aa
    return generated_sequence

In [27]:
# Define parameters
seed_sequence = 'MEKIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTH'  # Starting with Methionine (common start amino acid)
desired_length = 568  # Generate sequences of length 100
temperature = 0.8     # Adjust temperature to control dissimilarity

# Generate a sequence
generated_seq = generate_sequence(
    seed_sequence,
    model,
    aa_to_int,
    int_to_aa,
    max_sequence_length,
    desired_length,
    temperature
)

print(f"Generated sequence:\n{generated_seq}")

Generated sequence:
MEKIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQDILEKTHNGKLCDLNGVKPLILKDCSVAGWLLGNPMCDEFIRVPEWSYIVERANPTNDLCYPGNLNDYEELKHLLSRINHFEKTLIIPKSSWPNHETSLGVSAACPYQGAPSFFRNVVWLIKKNDAYPTIKISYNNTNREDLLILWGIHHSNNAEEQTNLYKNPTTYISVGTSTLNQRLVPKIATRSQVNGQRGRMDFFWTILKPDDAIHFESNGNFIAPEYAYKIVKKGDSTIMKSEVEYGHCNTKCQTPIGAINSSMPFHNIHPLTIGECPKYVKSNKLVLATGLRNSPLREKRRKRGLFGAIAGFIEGGWQGMVDGWYGYHHSNEQGSGYAADKESTQKAIDGVTNKVNSIIDKMNTQFEAVGREFNNLERRIENLNKKMEDGFLDVWTYNAELLVLMENERTLDFHDSNVKNLYDKVRLQLRDNAKELGNGCFEFYHKCDNECMESVRNGTYDYPQYSEEARLKREEISGVKLESIGTYQILSIYSTVASSLALAIMVAGLSLWMCSNGSLQCRICIK


In [31]:
import pickle

pickle.dump(model,open("rf.h5","wb"))

In [32]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def calculate_sequence_identity(seq1, seq2):
    # Perform global alignment
    alignments = pairwise2.align.globalxx(seq1, seq2)
    best_alignment = alignments[0]
    aligned_seq1, aligned_seq2, score, start, end = best_alignment
    # Calculate identity
    matches = sum(aa1 == aa2 for aa1, aa2 in zip(aligned_seq1, aligned_seq2))
    identity = matches / max(len(seq1), len(seq2)) * 100
    return identity

# Compare generated sequence with a sequence from the training set
training_seq = sequences[0]  # Assuming 'sequences' is your list of training sequences
identity = calculate_sequence_identity(generated_seq, training_seq)
print(f"Sequence identity with training sequence: {identity:.2f}%")

Sequence identity with training sequence: 95.06%


/opt/conda/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
